In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, json
import glob

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
temp = pd.DataFrame()

path_to_json = 'test_timelines/*' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

for file in file_list:
    data = pd.read_json(file, lines=True)
    data = pd.json_normalize(data['frames'])
    temp = temp.append(data, ignore_index = True)

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

...

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignore_index = True)
C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_22360\3914275407.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data, ignor

In [7]:
temp.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 1055, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'level': 2, 'participantId': 3, '...","{'events': [{'creatorId': 10, 'timestamp': 185...","{'events': [{'itemId': 1001, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'level': 6, 'participantId': 1, '...","{'events': [{'level': 5, 'participantId': 4, '...","{'events': [{'level': 7, 'participantId': 6, '...","{'events': [{'creatorId': 1, 'timestamp': 5464..."
1,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 1036, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'itemId': 2003, 'participantId': ...","{'events': [{'itemId': 2003, 'participantId': ...","{'events': [{'creatorId': 6, 'timestamp': 2403...","{'events': [{'itemId': 2003, 'participantId': ...","{'events': [{'itemId': 2003, 'participantId': ...","{'events': [{'creatorId': 4, 'timestamp': 4242...","{'events': [{'itemId': 2055, 'participantId': ...","{'events': [{'itemId': 1052, 'participantId': ..."
2,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 2033, 'participantId': ...","{'events': [{'creatorId': 1, 'timestamp': 6078...","{'events': [{'level': 2, 'participantId': 8, '...","{'events': [{'level': 3, 'participantId': 2, '...","{'events': [{'itemId': 2010, 'participantId': ...","{'events': [{'itemId': 1027, 'participantId': ...","{'events': [{'level': 7, 'participantId': 3, '...","{'events': [{'itemId': 2055, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'bounty': 0, 'killerId': 8, 'kill..."
3,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 1054, 'participantId': ...","{'events': [{'itemId': 1055, 'participantId': ...","{'events': [{'bounty': 400, 'killStreakLength'...","{'events': [{'itemId': 1036, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'itemId': 2010, 'participantId': ...","{'events': [{'killerId': 3, 'laneType': 'MID_L...","{'events': [{'bounty': 300, 'killStreakLength'...","{'events': [{'assistingParticipantIds': [8], '...","{'events': [{'itemId': 2003, 'participantId': ..."
4,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 1054, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'level': 2, 'participantId': 3, '...","{'events': [{'creatorId': 2, 'timestamp': 1804...","{'events': [{'creatorId': 2, 'timestamp': 2402...","{'events': [{'creatorId': 2, 'timestamp': 3002...","{'events': [{'assistingParticipantIds': [9], '...","{'events': [{'creatorId': 2, 'timestamp': 4201...","{'events': [{'itemId': 2055, 'participantId': ...","{'events': [{'creatorId': 2, 'timestamp': 5417..."
5,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'level': 2, 'participantId': 8, '...","{'events': [{'level': 2, 'participantId': 5, '...","{'events': [{'killerId': 0, 'laneType': 'BOT_L...","{'events': [{'itemId': 1083, 'participantId': ...","{'events': [{'level': 5, 'participantId': 4, '...","{'events': [{'killerId': 1, 'timestamp': 42069...","{'events': [{'itemId': 1018, 'participantId': ...","{'events': [{'creatorId': 1, 'timestamp': 5413..."
6,"{'events': [{'timestamp': 0, 'type': 'PAUSE_EN...","{'events': [{'itemId': 1036, 'participantId': ...","{'events': [{'levelUpType': 'NORMAL', 'partici...","{'events': [{'level': 2, 'participantId': 3, '...","{'events': [{'level': 3, 'participantId': 6, '...","{'events': [{'itemId': 1036, 'participantId': ...","{'events': [{'itemId': 3057, 'participantId': ...","{'events': [{'itemId': 2003, 'participantId': ...","{'events': [{'itemId': 1026, 'participantId': ...","{'events': [{'itemId': 2010, 'participantId': ...","{'events'

In [1]:
temp.to_csv('temp_test_timelines.csv')

NameError: name 'temp' is not defined